In [1]:
import sys
sys.path.append("../..")

In [9]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from models.src import train_model

In [18]:
from models.src import evaluate

In [4]:
model_list = ["GaussianNB","HistGradientBoostingClassifier",
                "AdaBoostClassifier",
                "RandomForestClassifier",
                "KNeighborsClassifier",
                "MLPClassifier",
                "SVC",
                "DecisionTreeClassifier"]
param_grid = [
    {}
]*8

In [5]:
outputs = []

In [6]:
for i in tqdm(range(len(model_list))):
    
    model_config = {"model_type": 'traditional', "clf": model_list[i], 
                   "name": f"{model_list[i]}_30sec"}
    data_config = {"use_file": "../../data/features_30_sec.csv", # so no preprocess
                   "split_shuffle": True,
                   "split_stratify": True,
                   "train_split": 0.7, 
                   "test_split": 0.15,
                   "seed": 2024, }
    train_config = {
                    "grid_search": param_grid[i], #'max_leaf_nodes': [30, 60, 120]},
                    "n_folds": 5, 
                    "metric": ['f1_macro', 'accuracy'], 
                    "seed": 2024,
                    "save_location": "../output"
                    }
    outputs.append(train_model.train(model_config, data_config, train_config,))

 25%|██▌       | 2/8 [00:22<01:18, 13.08s/it]/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warni

In [11]:
test_results = pd.DataFrame([outputs[i].results for i in range(len(model_list))])
test_results['model'] = model_list

In [20]:
cv_results = pd.DataFrame([outputs[i].search.cv_results_ for i in range(len(model_list))])
cv_results['model'] = model_list

In [21]:
cv_results = explode(cv_results, list(set(cv_results.columns)-{'model'}))

In [22]:
cv_results

,model,split3_test_f1_macro,mean_score_time,split2_test_f1_macro,split2_test_accuracy,rank_test_f1_macro,mean_test_f1_macro,std_test_f1_macro,std_test_accuracy,std_fit_time,...,split4_test_f1_macro,split3_test_accuracy,rank_test_accuracy,params,mean_fit_time,mean_test_accuracy,split1_test_f1_macro,std_score_time,split1_test_accuracy,split0_test_f1_macro
0,GaussianNB,0.640115,0.005059,0.581070,0.588235,1,0.593931,0.028306,0.029923,0.000390,...,0.553832,0.653595,1,{},0.004018,0.602614,0.603516,0.000139,0.607843,0.591121
1,HistGradientBoostingClassifier,0.723951,0.018393,0.765992,0.771242,1,0.727528,0.030049,0.028936,0.027767,...,0.689971,0.725490,1,{},3.431799,0.732026,0.757220,0.000382,0.758170,0.700504
2,AdaBoostClassifier,0.144880,0.014144,0.091625,0.163399,1,0.153824,0.037874,0.045093,0.012908,...,0.187498,0.228758,1,{},0.795372,0.241830,0.198899,0.000467,0.274510,0.146216
3,RandomForestClassifier,0.683385,0.010462,0.673525,0.686275,1,0.705402,0.022405,0.018943,0.016966,...,0.728581,0.692810,1,{},0.622036,0.712418,0.720794,0.000337,0.725490,0.720724
4,KNeighborsClassifier,0.447306,0.015278,0.374467,0.385621,1,0.435356,0.038196,0.039778,0.000426,...,0.474083,0.464052,1,{},0.002804,0.447059,0.409699,0.008080,0.418301,0.471223
5,MLPClassifier,0.341823,0.010841,0.377758,0.431373,1,0.353256,0.015300,0.031373,0.102666,...,0.344124,0.352941,1,{},0.534947,0.388235,0.338090,0.000381,0.379085,0.364483
6,SVC,0.339520,0.023605,0.299745,0.339869,1,0.344068,0.026127,0.028339,0.000929,...,0.362360,0.398693,1,{},0.038193,0.385621,0.341667,0.000853,0.372549,0.377046
7,DecisionTreeClassifier,0.396290,0.004149,0.479267,0.477124,1,0.480870,0.048146,0.050187,0.004814,...,0.487996,0.392157,1,{},0.098907,0.478431,0.495318,0.000078,0.483660,0.545478
